## Visión por Computador

**Grupo 17**:

- Sara Expósito Suárez
- Alejandro Padrón Ossorio

Paquetes necesarios

In [2]:
import cv2  
import numpy as np
import matplotlib.pyplot as plt

TAREA: Captura una o varias imágenes con monedas no solapadas. Tras visualizar la imagen, identifica de forma interactiva (por ejemplo haciendo clic en la imagen) una moneda de un valor determinado en la imagen (por ejemplo de 1€). Tras ello, la tarea se resuelve mostrando por pantalla el número de monedas y la cantidad de dinero presentes en la imagen. No hay restricciones sobre utilizar medidas geométricas o de color. ¿Qué problemas han observado?

Nota: Para establecer la correspondencia entre píxeles y milímetros, comentar que la moneda de un euro tiene un diámetro de 23.25 mm. la de 50 céntimos de 24.35, la de 20 céntimos de 22.25, etc. 

Extras: Considerar que la imagen pueda contener objetos que no son monedas y/o haya solape entre las monedas. Demo en vivo.



96.73095703125
Diametro: 96.73095703125 Cantidad: 1
Diametro: 102.7784194946289 Cantidad: 1.5
Diametro: 107.3297119140625 Cantidad: 3.5
Diametro: 69.71328735351562 Cantidad: 3.51
Diametro: 78.80816650390625 Cantidad: 3.53
Diametro: 85.33612823486328 Cantidad: 3.5799999999999996
Diametro: 88.52730560302734 Cantidad: 3.6299999999999994
Diametro: 94.68267822265625 Cantidad: 3.8299999999999996


In [67]:
img = cv2.imread('Monedas2.jpg')
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_gris = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)

img_ggris = cv2.GaussianBlur(img_gris, (11, 11), 0)

th,img_th = cv2.threshold(img_ggris,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

contornos, hierarchy = cv2.findContours(img_th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

cv2.drawContours(img_rgb, contornos, -1, (0, 255, 0), 3)

radios = []

selected_radius = None

def select_coin(event, x, y, flags, param):
    global selected_radius
    if event == cv2.EVENT_LBUTTONDOWN:
        for c in contornos:
            (cx, cy), radio = cv2.minEnclosingCircle(c)
            if cv2.pointPolygonTest(c, (x, y), False) >= 0:
                selected_radius = radio
                break

cv2.imshow("Selecciona la moneda de 1 euro", img)
cv2.setMouseCallback("Selecciona la moneda de 1 euro", select_coin)

while selected_radius is None:
    if cv2.waitKey(10) == 27:
        break

for c in contornos:
    area = cv2.contourArea(c)
    if area > 10:
        (cx, cy), radio = cv2.minEnclosingCircle(c)
        radios.append(radio)

coins_diameter = [16.26, 18.75, 21.25, 19.75, 22.25, 24.25, 23.25, 25.75]
coins_value = [0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1, 2]

if selected_radius:
    ratio = selected_radius * 2 / coins_diameter[6]

money_amount = 0
tolerance = 4
print(selected_radius * 2)

for radio in radios:
    diameter = radio * 2
    detected = False
    
    for coin_diameter, coin_value in zip(coins_diameter, coins_value):
        if abs(diameter - ratio * coin_diameter) < tolerance:
            money_amount += coin_value
            detected = True
            break

    if not detected:
        money_amount += coins_value[-1]
    
    print("Diametro:", diameter, "Cantidad:", money_amount)

# print(money_amount)
plt.imshow(img_rgb)
plt.axis("off")

cv2.destroyAllWindows()

TAREA: Las tres imágenes cargadas en la celda inicial, han sido extraidas de las imágenes de mayor tamaño presentes en la carpeta. La tarea consiste en extraer características (geométricas y/o visuales) e identificar patrones que permitan distinguir las partículas de cada una de las tres clases, evaluando los aciertos y fallos con las imágenes completas considerando las métricas mostradas y la matriz de confusión. La matriz de confusión, muestra para cada clase el número de muestras que se clasifican correctamente de dicha clase, y el número de muestras que se clasifican incorrectamente por cada una de las otras dos clases.

En el trabajo [SMACC: A System for Microplastics Automatic Counting and Classification](https://doi.org/10.1109/ACCESS.2020.2970498), las características geométricas utilizadas fueron:

- Área en píxeles
- Perímetro en píxeles
- Compacidad (relación entre el cuadrado del perímetro y el área de la partícula)
- Relación del área de la partícula con la del contenedor
- Relación del ancho y el alto del contenedor
- Relación entre los ejes de la elipse ajustada
- Definido el centroide, relación entre las distancias menor y mayor al contorno